In [ ]:
import sys
sys.path.append('./concurrent_set')  # tùy thuộc vào đường dẫn thư mục thực tế

from fine_grained_set import FineGrainedSet


ModuleNotFoundError: No module named 'fine_grained_set'

In [ ]:
class NodeWithPriority:
    def __init__(self, value, priority):
        self.value = value
        self.priority = priority
    
    def __lt__(self, other):  # để so sánh trong danh sách
        return self.priority < other.priority

class ConcurrentPriorityQueue:
    def __init__(self):
        self.set = FineGrainedSet()
        self.items = {}

    def insert(self, node, priority):
        wrapped = NodeWithPriority(node, priority)
        self.items[node] = wrapped
        self.set.add(wrapped)

    def pop_min(self):
        # Lấy phần tử đầu tiên (có priority nhỏ nhất)
        curr = self.set.head.next
        if curr == self.set.tail:
            return None
        self.set.remove(curr.value)
        return curr.value.value  # Trả lại node gốc

    def is_empty(self):
        return self.set.head.next == self.set.tail


In [ ]:
def heuristic(a, b):
    coord_a = (G.nodes[a]['y'], G.nodes[a]['x'])
    coord_b = (G.nodes[b]['y'], G.nodes[b]['x'])
    return geodesic(coord_a, coord_b).meters

def parallel_a_star(G, start, goal, num_threads=4):
    open_set = ConcurrentPriorityQueue()
    open_set.insert(start, 0)
    
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}
    came_from = {}
    visited = set()
    lock = threading.Lock()

    def worker():
        while not open_set.is_empty():
            with lock:
                current = open_set.pop_min()
            if current is None:
                return

            if current == goal:
                return

            for neighbor in G.successors(current):
                tentative_g = g_score.get(current, float('inf')) + G.edges[current, neighbor, 0].get("travel_time", 1)
                if tentative_g < g_score.get(neighbor, float('inf')):
                    with lock:
                        g_score[neighbor] = tentative_g
                        f_score[neighbor] = tentative_g + heuristic(neighbor, goal)
                        came_from[neighbor] = current
                        open_set.insert(neighbor, f_score[neighbor])

    threads = [threading.Thread(target=worker) for _ in range(num_threads)]
    for t in threads: t.start()
    for t in threads: t.join()

    return came_from


In [ ]:
def reconstruct_path(came_from, current):
    path = [current]
    while current in came_from:
        current = came_from[current]
        path.append(current)
    path.reverse()
    return path

start_time = time.time()
came_from = parallel_a_star(G, start_node, goal_node, num_threads=4)
end_time = time.time()

path = reconstruct_path(came_from, goal_node)
print(f"🔄 Path length: {len(path)} nodes")
print(f"⏱ Execution time: {end_time - start_time:.4f}s")
